In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## importing necessary libries 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

### **Reading CSV File**

In [ ]:
bread_df = pd.read_csv('/kaggle/input/the-bread-basket/bread basket.csv')

#### viewing top five rows

In [ ]:
#viewing top 5 rows 
bread_df.head()

#### checking the shape of data

In [ ]:
bread_df.shape

#### checking for null values

In [ ]:
bread_df.isna().sum()#checking null values

#### displaying 5 number summary

In [ ]:
bread_df.describe(include='all')#checking 5 number summary

#### checking the data type of columns

In [ ]:
bread_df.dtypes

#### checking the the unique item count

In [ ]:
bread_df['Item'].nunique()

In [ ]:
bread_df.head(1)

In [ ]:
bread_df['date_time'] = pd.to_datetime(bread_df['date_time'])

#### checking the unique transactions

In [ ]:
bread_df['Transaction'].nunique()

In [ ]:
bread_df.head(2)

#### checking count by peroid_day and displaying percentage by it

In [ ]:
sns.countplot(x='period_day',data=bread_df)
plt.show()
print("Percentage Wise Count\n",bread_df['period_day'].value_counts(normalize=True)*100)

In [ ]:
sns.countplot(x='weekday_weekend',data=bread_df)
plt.show()
print("Percentage Wise Count\n",bread_df['weekday_weekend'].value_counts(normalize=True)*100)

In [ ]:
bread_df['time'] = bread_df['date_time'].dt.time

In [ ]:
bread_df['hour'] = bread_df['date_time'].dt.hour

In [ ]:
bread_df['month'] = bread_df['date_time'].dt.month

In [ ]:
bread_df.head()

In [ ]:

# Replacing hours with text
hour_in_num = (1,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23)
hour_in_obj = ('1-2','7-8','8-9','9-10','10-11','11-12','12-13','13-14','14-15',
               '15-16','16-17','17-18','18-19','19-20','20-21','21-22','22-23','23-24')
bread_df['hour'] = bread_df['hour'].replace(hour_in_num, hour_in_obj)

# Extracting weekday and replacing it with text
bread_df['weekday'] = bread_df['date_time'].dt.weekday
bread_df['weekday'] = bread_df['weekday'].replace((0,1,2,3,4,5,6), 
                                          ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'))

# dropping date_time column
bread_df.drop('date_time', axis = 1, inplace = True)

In [ ]:
bread_df.head()

In [ ]:
bread_df['month'] = bread_df['month'].replace((1,2,3,4,5,6,7,8,9,10,11,12), 
                                          ('January','February','March','April','May','June','July','August',
                                          'September','October','November','December'))

In [ ]:
bread_df.head()

In [ ]:
bread_df['Item'] = bread_df['Item'].apply(lambda x:x.strip())

In [ ]:
bread_df['Item'] = bread_df['Item'].apply(lambda x:x.lower())

In [ ]:
bread_df.head(2)

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x = bread_df['Item'].value_counts().head(20).index,y=bread_df['Item'].value_counts().head(20).values, palette = 'gnuplot')
plt.xlabel('Items', size = 15)
plt.xticks(rotation=45)
plt.ylabel('Count of Items', size = 15)
plt.title('Top 20 Items purchased by customers', color = 'green', size = 20)
plt.show()

In [ ]:
month_trans = bread_df.groupby('month')['Transaction'].count().reset_index()
month_trans.loc[:,"monthorder"] = [4,8,12,2,1,7,6,3,5,11,10,9]
plt.figure(figsize=(12,8))
sns.barplot(x='month',y='Transaction',data=bread_df)
plt.xlabel('Month Name')
plt.ylabel('Count')
plt.title('Count Per Month')
plt.show()

In [ ]:
month_trans = bread_df.groupby('weekday')['Transaction'].count().reset_index()
month_trans.loc[:,"weekorder"] = [4,0,5,6,3,1,2]
plt.figure(figsize=(12,8))
sns.barplot(x='weekday',y='Transaction',data=bread_df)
plt.xlabel('weekday Name')
plt.ylabel('Count')
plt.title('Count Per weekday')
plt.show()

In [ ]:
hourTran = bread_df.groupby('hour')['Transaction'].count().reset_index()
hourTran.loc[:,"hourorder"] = [1,10,11,12,13,14,15,16,17,18,19,20,21,22,23,7,8,9]
hourTran.sort_values("hourorder",inplace=True)

plt.figure(figsize=(12,5))
sns.barplot(data = bread_df, x = "Transaction", y = "hour")
plt.ylabel('Hours', size = 15)
plt.xlabel('Orders each hour', size = 15)
plt.title('Count of orders received each hour', color = 'green', size = 20)
plt.show()

In [ ]:
bread_df.groupby(['period_day','Item']).count().reset_index().sort_values(['period_day','Transaction'],ascending=False)
day = ['morning','afternoon','evening','night']

plt.figure(figsize=(15,8))
for i,j in enumerate(day):
    plt.subplot(2,2,i+1)
    df1 = bread_df[bread_df.period_day==j].head(10)
    sns.barplot(data=df1, y=df1.Item, x=df1.Transaction, color='pink')
    plt.xlabel('')
    plt.ylabel('')
    plt.title('Top 10 items people like to order in "{}"'.format(j), size=13)

plt.show()

In [ ]:
transactions_str = bread_df.groupby(['Transaction', 'Item'])['Item'].count().reset_index(name ='Count')
transactions_str


In [ ]:
from mlxtend.frequent_patterns import association_rules, apriori

In [ ]:
my_basket = transactions_str.pivot_table(index='Transaction', columns='Item', values='Count', aggfunc='sum').fillna(0)
my_basket.tail()

In [ ]:
def encode(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

In [ ]:
my_basket_sets = my_basket.applymap(encode)
my_basket_sets.tail()

In [ ]:
# using the 'apriori algorithm' with min_support=0.01 (1% of 9465)
# It means the item should be present in atleast 94 transaction out of 9465 transactions only when we considered that item in
# frequent itemset
frequent_items = apriori(my_basket_sets, min_support = 0.01,use_colnames = True)
frequent_items

In [ ]:
ass_rule =  association_rules(frequent_items,metric='lift',min_threshold=1)

In [ ]:
ass_rule.sort_values(by='confidence',ascending=False)